In [ ]:
import sys
sys.path[0] = ('/home/labs/waic/omrik/DNN-Challenge')
from fastai.vision import *
import pre
import resample

root = Path('../data').resolve()
train = root / 'train'
val = root / 'val'

In [ ]:
def normalize_time(series):
    # 1440 minutes in a day
    normalized = (series.hour * 60 + series.minute) / 1440
    return normalized

def get_data(data_dir, sub_mean=False):
    cgm, meals = pre.get_dfs(data_dir)
    if sub_mean:
        mean, std = pre.norm_stats['GlucoseValue']
        cgm['GlucoseValue'] = cgm['GlucoseValue'] - mean / std
        
    meals = resample.resample_meals(cgm, meals, 15)
    meals = pd.concat((meals, cgm), axis=1)
    meals['time'] = normalize_time(meals.index.get_level_values('Date'))
    cgm, y = pre.build_cgm(cgm)
    return cgm, meals, y

class ContData(Dataset):
    def __init__(self, cgm, meals, y):
        self.cgm = cgm
        self.meals = meals
        self.y = y
        
    def __len__(self):
        return len(self.cgm)
    
    def __getitem__(self, i):
        index = self.meals.index.get_loc(self.cgm.index[i])
        values = self.meals[index-48:index+1].values
        target = self.y.iloc[i].values
        x, y = torch.tensor(values, dtype=torch.float), torch.tensor(target, dtype=torch.float)
        return x, y
    
class AddNoise(Callback):
    
    def __init__(self, std):
        self.std = std
        
    def on_batch_begin(self, last_input, last_target, train, **kwargs):
        if not train:
            return
        
        size = last_input.shape[0], last_input.shape[2]
        noise = torch.normal(0, self.std, size).to(last_input.device)
        last_input[:, -2] += noise
        return {'last_input': last_input}


In [ ]:
train_data = get_data(train)
val_data = get_data(val)
train_data[1].head()

In [ ]:
train_ds = ContData(*train_data)
val_ds = ContData(*val_data)
data = DataBunch.create(train_ds, val_ds, bs=512)
data

In [ ]:
from metrics import Pearson


class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Linear(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input)
        output = embedded
        output, hidden = self.gru(output[None], hidden)
        return output[0], hidden

    def initHidden(self, bs, device):
        return torch.zeros(1, bs, self.hidden_size, device=device)


In [ ]:
MAX_LENGTH = 49

class AttnDecoder(nn.Module):
    def __init__(self, hidden_size, output_size=8, max_length=MAX_LENGTH):
        super().__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.max_length = max_length

        self.attn = nn.Linear(self.hidden_size, 1)
        self.attn_combine = nn.Linear(self.hidden_size, self.hidden_size)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, encoder_outputs):
        attn_weights = F.softmax(self.attn(encoder_outputs).squeeze(), dim=1)
        attn_applied = torch.bmm(attn_weights[:, None], encoder_outputs).squeeze()

        output = self.attn_combine(attn_applied)
        output = F.relu(output)

        output = self.out(output)
        return output

In [ ]:
class Seq2Seq(Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.encoder = EncoderRNN(input_size, hidden_size)
        self.decoder = AttnDecoder(hidden_size)
        
    def forward(self, input):
        device = input.device
        bs = input.shape[0]
        input = input.transpose(0, 1)
        
        encoder_hidden = self.encoder.initHidden(bs, device)
        encoder_outputs = input.new_zeros(bs, MAX_LENGTH, self.encoder.hidden_size)
        
        for ei in range(input.shape[0]):
            encoder_output, encoder_hidden = self.encoder(input[ei], encoder_hidden)
            encoder_outputs[:, ei] = encoder_output
            
            
        out = self.decoder(encoder_outputs)
        return out

In [ ]:
defaults.device = torch.device('cpu')
model = Seq2Seq(38, 128)
metrics = [mean_absolute_error, Pearson(val_ds.y)]
learner = Learner(data, model, loss_func=nn.MSELoss(), metrics=metrics)

In [ ]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(10, 1e-3)

In [ ]:
learner.save('gru1-attn-h128')

In [ ]:
train_data = get_data(train, sub_mean=True)
val_data = get_data(val, sub_mean=True)

In [ ]:
train_ds = ContData(*train_data)
val_ds = ContData(*val_data)
data = DataBunch.create(train_ds, val_ds, bs=512)
data

In [ ]:
model = Seq2Seq(38, 128)
metrics = [mean_absolute_error, Pearson(val_ds.y)]
learner = Learner(data, model, loss_func=nn.MSELoss(), metrics=metrics)

In [ ]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(10, 1e-3)

## with noise

In [ ]:
model = Seq2Seq(38, 128)
metrics = [mean_absolute_error, Pearson(val_ds.y)]
learner = Learner(data, model, loss_func=nn.MSELoss(), metrics=metrics, callbacks=AddNoise(0.1))

In [ ]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(10, 1e-3)

## Seq2Lin

In [ ]:
train_data = get_data(train)
val_data = get_data(val)
train_ds = ContData(*train_data)
val_ds = ContData(*val_data)
data = DataBunch.create(train_ds, val_ds, bs=512)
data

In [ ]:
class Seq2Lin(Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        self.encoder = EncoderRNN(input_size, hidden_size)
        self.self_attention = SelfAttention(MAX_LENGTH) if self_attention else None
        self.relu = nn.ReLU(inplace=True)
        self.decoder = nn.Linear(hidden_size, 8)

        
    def forward(self, input):
        device = input.device
        bs = input.shape[0]
        input = input.transpose(0, 1)
        
        encoder_hidden = self.encoder.initHidden(bs, device)
        encoder_outputs = input.new_zeros(bs, MAX_LENGTH, self.encoder.hidden_size)
        
        for ei in range(input.shape[0]):
            encoder_output, encoder_hidden = self.encoder(input[ei], encoder_hidden)
            encoder_outputs[:, ei] = encoder_output

        encoder_outputs = self.self_attention(encoder_outputs)
        out = encoder_outputs.permute(1, 2, 0)
        out = self.lin1(out)
        out = self.relu(out)
        out = self.lin2(encoder_outputs.view(bs, -1))
        return out

In [ ]:
model = Seq2Lin(38, 128)
metrics = [mean_absolute_error, Pearson(val_ds.y)]
learner = Learner(data, model, loss_func=nn.MSELoss(), metrics=metrics, callbacks=AddNoise(0.1))

In [ ]:
learner.lr_find()
learner.recorder.plot(suggestion=True)

In [ ]:
learner.fit_one_cycle(10, 1e-3)